<a href="https://colab.research.google.com/github/sberniz/DS-Unit-1-Sprint-2-Statistics/blob/master/module2/LS_DS_122_Chi2_Tests_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment - Practice Chi^2 Tests

Use the following dataset relating to math scores of students in two different Portugese schools:

<https://archive.ics.uci.edu/ml/datasets/Student+Performance>

### 1) Load the dataset specifically relating to math scores as a new dataframe.

There are two datasets in the `student.zip` file, make sure you use `student-mat.csv`.


In [0]:
import pandas as pd
import numpy as np
from scipy import stats

In [0]:
# YOUR WORK HERE
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip

--2020-06-09 19:01:59--  https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20478 (20K) [application/x-httpd-php]
Saving to: ‘student.zip’

student.zip         100%[===================>]  20.00K  --.-KB/s    in 0.1s    

2020-06-09 19:02:00 (154 KB/s) - ‘student.zip’ saved [20478/20478]



In [0]:
!unzip student.zip

Archive:  student.zip
  inflating: student-mat.csv         
  inflating: student-por.csv         
  inflating: student-merge.R         
  inflating: student.txt             


In [0]:
df = pd.read_csv('student-mat.csv',sep=';')
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


### 2) Use Chi^2 tests and `stats.chi2_contingency()` to identify:
 - Two pairs of variables that are dependent (are associated with one another).
 - Two pairs of variables that are independent (have no significant relationship).

Does it make intuitive sense why the variables in these pairs might or might not show a relationship to one another? 


In [86]:
contingency3 = pd.crosstab(df['absences'],df['failures'])
contingency3

failures,0,1,2,3
absences,,,,
0,89,14,5,7
1,3,0,0,0
2,58,4,1,2
3,6,1,0,1
4,47,4,2,0
5,4,0,0,1
6,25,3,2,1
7,6,1,0,0
8,17,2,2,1


In [89]:
#high stats and near 0 #dependent variable suggested, null hypothesis rejected. 
chi2, p_value,dof,expected = stats.chi2_contingency(contingency3)
print("chi2 stats", chi2)
print("p_value",p_value)
print("dof",dof)
print("expected",expected)

chi2 stats 122.18899006084001
p_value 0.05694057065252943
dof 99
expected [[9.08354430e+01 1.45569620e+01 4.94936709e+00 4.65822785e+00]
 [2.36962025e+00 3.79746835e-01 1.29113924e-01 1.21518987e-01]
 [5.13417722e+01 8.22784810e+00 2.79746835e+00 2.63291139e+00]
 [6.31898734e+00 1.01265823e+00 3.44303797e-01 3.24050633e-01]
 [4.18632911e+01 6.70886076e+00 2.28101266e+00 2.14683544e+00]
 [3.94936709e+00 6.32911392e-01 2.15189873e-01 2.02531646e-01]
 [2.44860759e+01 3.92405063e+00 1.33417722e+00 1.25569620e+00]
 [5.52911392e+00 8.86075949e-01 3.01265823e-01 2.83544304e-01]
 [1.73772152e+01 2.78481013e+00 9.46835443e-01 8.91139241e-01]
 [2.36962025e+00 3.79746835e-01 1.29113924e-01 1.21518987e-01]
 [1.34278481e+01 2.15189873e+00 7.31645570e-01 6.88607595e-01]
 [2.36962025e+00 3.79746835e-01 1.29113924e-01 1.21518987e-01]
 [9.47848101e+00 1.51898734e+00 5.16455696e-01 4.86075949e-01]
 [2.36962025e+00 3.79746835e-01 1.29113924e-01 1.21518987e-01]
 [9.47848101e+00 1.51898734e+00 5.16455696e-

In [0]:
df['schoolsup'].value_counts().sort_index()

no     344
yes     51
Name: schoolsup, dtype: int64

In [0]:
#dependent variable
df['freetime'].value_counts().sort_index()

1     19
2     64
3    157
4    115
5     40
Name: freetime, dtype: int64

In [0]:
contingency1 = pd.crosstab(df['schoolsup'],df['freetime'])
contingency1

freetime,1,2,3,4,5
schoolsup,,,,,
no,13,60,133,103,35
yes,6,4,24,12,5


In [0]:
#dependent if they have school support is less free time 
chi2,p_value,dof,expected = stats.chi2_contingency(contingency1)
print("Chi statistic", chi2)
print("P value",p_value)
print("degrees of freedom",dof)
print("expected", expected)

Chi statistic 9.835074983378254
P value 0.04329937433691196
degrees of freedom 4
expected [[ 16.54683544  55.73670886 136.72911392 100.15189873  34.83544304]
 [  2.45316456   8.26329114  20.27088608  14.84810127   5.16455696]]


In [83]:
#Second pair of independent variable 
contingency2 = pd.crosstab(df['failures'],df['Pstatus'])
contingency2

Pstatus,A,T
failures,,
0,33,279
1,4,46
2,2,15
3,2,14


In [84]:
#P_value closer to 1, not close to 0, "assume " null hypothesis (Independent value) fail to reject null hypothesis.
chi2,p_value,dof, expected = stats.chi2_contingency(contingency2)
print("Chi2 stats",chi2)
print("p_value",p_value)
print("Degree of freedom",dof)
print ('expected',expected)

Chi2 stats 0.4298108254972419
p_value 0.9340177102114694
Degree of freedom 3
expected [[ 32.38481013 279.61518987]
 [  5.18987342  44.81012658]
 [  1.76455696  15.23544304]
 [  1.66075949  14.33924051]]


In [0]:
# YOUR WORK HERE
#Pair that are independent (Not Aassociated with one another) (low statistic, high p-value)
df['romantic'].value_counts().sort_index()

no     263
yes    132
Name: romantic, dtype: int64

In [0]:
df['goout'].value_counts().sort_index()

1     23
2    103
3    130
4     86
5     53
Name: goout, dtype: int64

In [0]:
contingency = pd.crosstab(df['romantic'], df['goout'])
contingency

goout,1,2,3,4,5
romantic,,,,,
no,16,67,87,60,33
yes,7,36,43,26,20


In [0]:
chi2,p_value,dof,expected = stats.chi2_contingency(contingency)
print('Chi2 statistic', chi2)
print("P value",p_value)
print("degree of freedom",dof)
print("expected frequencies table", expected)

Chi2 statistic 1.043935158800679
P value 0.9030624031255541
degree of freedom 4
expected frequencies table [[15.31392405 68.57974684 86.55696203 57.26075949 35.28860759]
 [ 7.68607595 34.42025316 43.44303797 28.73924051 17.71139241]]


### 3) Use NumPy to perform your own chi^2 test "from scratch" 

Pick any of the chi^2 tests that you ran in part 2 and try them on your own. You should get the same results that Scipy got for all four values returned from `chi2_contingency()`

In [0]:
# YOUR WORK HERE
#Observed values
#free time and schoolsup 
observed = pd.crosstab(df['schoolsup'],df['freetime'])
observed

freetime,1,2,3,4,5
schoolsup,,,,,
no,13,60,133,103,35
yes,6,4,24,12,5


In [0]:
#Expected
observed_with_margins = pd.crosstab(df['schoolsup'],df['freetime'],margins=True)
observed_with_margins

freetime,1,2,3,4,5,All
schoolsup,,,,,,
no,13,60,133,103,35,344
yes,6,4,24,12,5,51
All,19,64,157,115,40,395


In [0]:
#expected_Row_col 
expected_row_col = 344 * 19 / 395
expected_row_col

16.546835443037974

In [0]:
observed_values = observed.values
observed_values

array([[ 13,  60, 133, 103,  35],
       [  6,   4,  24,  12,   5]])

In [0]:
observed[1][1]

6

In [0]:
row_totals = df['schoolsup'].value_counts().sort_index().values
row_totals

array([344,  51])

In [0]:
col_totals = df['freetime'].value_counts().sort_index().values
col_totals

array([ 19,  64, 157, 115,  40])

In [0]:
sample_size = df.shape[0]
sample_size #395

395

In [0]:
#calculate expectd value 
expected_list = []

for i, row_tot in enumerate(row_totals):
  row = []
  for j, col_tot in enumerate(col_totals):
    expected_value = ((row_tot*col_tot) / sample_size)
    row.append(expected_value)
  expected_list.append(row)
expected_list

[[16.546835443037974,
  55.73670886075949,
  136.72911392405064,
  100.15189873417721,
  34.835443037974684],
 [2.4531645569620255,
  8.263291139240506,
  20.270886075949367,
  14.848101265822784,
  5.1645569620253164]]

In [0]:
expected = np.array(expected_list)
expected #expected numpy array, confirmed with scipy is the same 

array([[ 16.54683544,  55.73670886, 136.72911392, 100.15189873,
         34.83544304],
       [  2.45316456,   8.26329114,  20.27088608,  14.84810127,
          5.16455696]])

In [0]:
#formula to calculate the chi2 
chi2 = (((observed_values - expected)**2) /expected).sum()
print("chi 2: ",chi2) #chi2 is the same. 

chi 2:  9.835074983378254


In [0]:
#calculate degrees of freedom
#(num of columns -1) * (num of rows -1)
dof = (observed.shape[1]  - 1)*(observed.shape[0] - 1)
print("DOF:",dof)

DOF: 4


In [0]:
p_value = 0.0433
print("P_value",p_value) #From the uiowa applet

P_value 0.0433


## Stretch goals:

### 1. Refactor your code so it is elegant, readable, and holds reusable code in functions.

In [0]:
# YOUR WORK HERE



### 2. Check For Understanding - Study and write your own explanations/definitions for these topics:

- What is a sample "estimate" in statistics?

- What are hypothesis test? How are they useful?

- What is a "null hypothesis?"

- What is a p-value and what does it represent?

- What does it mean for something to be "statistically significant?"

- What is a test statistic and how does it relate to a p-value?

- What are "degrees of freedom" and how are they calculated in a 1-sample, 2-sample, and chi^2 test for independence?

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)